In [1]:
%matplotlib inline

import pandas as pd 
import numpy as np
import itertools

import tokenize_uk
import cld3
import pymorphy2

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report, confusion_matrix

from sklearn.utils import resample

import matplotlib.pyplot as plt

In [2]:
df = pd.read_json('reviews.json')

df['text'] = df['text'].apply(lambda x: ' '.join(x))
df['lang'] = df['text'].apply(lambda x: cld3.get_language(x)[0])

#### Distribution among classes:

In [3]:
df.groupby(['stars'])['text'].count()

stars
1     2305
2     2304
3     4676
4    15051
5    42405
Name: text, dtype: int64

In [4]:
df.groupby(['lang']).count().reset_index().sort_values(['stars'], ascending=False)

,lang,stars,text
36,ru,55852,55852
43,uk,9871,9871
3,bg,261,261
2,be,184,184
38,sr,117,117
29,mn,101,101
28,mk,74,74
24,ky,61,61
42,tg,44,44
22,kk,35,35


##### CLD detected other languages except ru and uk, let's see what are those
Some of them are full of numbers and abbreviations, some of them are legit reviews: 

In [5]:
df.query('lang in ("lb", "ro", "ku", "jv", "is", "nl", "su", "id", "ht", "et", "xh", "eo", "sv", "hu", "es", "az")')

,stars,text,lang
653,5,Супер миксер!!!,eo
4439,4,Чайник как чайник,et
7867,3,проработал ровно год.,ro
11498,5,Це фул хд?,ht
12947,4,"CrystalDiskMark test : Read 240 MB/s, Write 23...",hu
15886,4,Какое разшерение экрана? FHD?,lb
18598,4,MSI Z370 gaming plus?,id
18618,5,Совместим ли с Asus Z170-A?,es
22307,5,Мыщь,sv
29168,4,Для её цены хорошая мышь,is


Some transliteration:

In [6]:
df.query('lang in ("ru-Latn")')

,stars,text,lang
3751,5,Prekrasnaya i praktichnaya mashyna. Protestiro...,ru-Latn
31156,4,"Otli4nie ushi, skuzhat uzhe 6 let, dazhe sey4a...",ru-Latn
31736,4,"Pol'zovalsja porta pro i etimi, membrani poxoz...",ru-Latn
39266,4,А откуда идёт посылка?,ru-Latn
39490,3,slomalis cherez 2 nedeli v sumke Достоинства: ...,ru-Latn
41563,4,Horoshaya model'. Oni nebol'shie po sravneniyu...,ru-Latn
47502,4,"poka rabotayut Достоинства: horoshiy zvuk, udo...",ru-Latn
51107,4,vse nravitsya!,ru-Latn
64278,5,Потянет gigabyte gtx 1060 6gb. Extreme Gaming?...,ru-Latn


Here are some very clear 5 star reviews (which are detected as `be` I think due to the length of string) 

In [7]:
df.query('lang in ("be")')

,stars,text,lang
395,5,супер,be
973,5,Супер!),be
1613,5,супер,be
1944,5,Супер!,be
2219,5,Супер,be
2317,5,Супер,be
3446,5,Какая разница между х700 и х710?,be
3872,5,супер,be
4239,5,Качество выше цены! Достоинства: Качество выше...,be
4280,5,Классный чайник за такую цену!!!,be


In [8]:
df.query('lang in ("mn")')

,stars,text,lang
734,5,Кухонный комбайн PHILIPS просто СУПЕР!!!Советую,mn
856,5,Хороший,mn
1262,5,Хороший миксер!,mn
1466,5,Хороший блендер!,mn
1534,5,мощный,mn
1901,5,Отличный кухонный комбайн.,mn
2803,5,"Хороший МФУ, оправдал ожидания",mn
2937,5,хороший принтер,mn
3589,5,"хороший аппарат, доволен",mn
3949,5,Хороший чайник,mn


If we filter out all those and take only ru and uk reviews, distibution is still more or less same as initial. 1, 2 and 3 stars reviews 
weren't filtered out. 

In [9]:
data = df.query('lang in ("uk", "ru")')
print('Initial:', len(df), 'Filtered:', len(data))

data.groupby(['stars'])['text'].count()

Initial: 66741 Filtered: 65723


stars
1     2298
2     2297
3     4642
4    14870
5    41616
Name: text, dtype: int64

#### Feature engineering 
First, I'll try to use inly reviews with correctly detected language

I found that some reviews aren't reviews but simple questions, like: 
review `Страна производитель?` with 5 stars. I think those may need special treatment

In [10]:
def is_question(text): 
    return text.strip().endswith('?')
    
        
data['is_question'] = data['text'].apply(lambda x: is_question(x))
data[(data['is_question'] == True) & (data['stars'] == 5)]

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,stars,text,lang,is_question
1,5,Вопрос к представителю:. Можно ли надеяться на...,ru,True
7,5,Майнкрафт на минимальных настройках графики по...,ru,True
8,5,Adobe P и Adobe I как пойдут?,ru,True
32,5,"Доброго времени суток. Подскажите пожалуйста, ...",ru,True
66,5,"Добрый день. Подскажите пожалуйста, в данной м...",ru,True
565,5,Очень довольны покупкой!но вопрос Можно ли им ...,ru,True
614,5,"Блендер дуже хороший, вся сім'я залишилась рад...",uk,True
660,5,"Добрый день! Сегодня получила эту модель, под...",ru,True
666,5,"Добрый день,подскажите прошу гарантия есть на...",ru,True
698,5,Я очень рада что купила именно этот блендер! Д...,ru,True


In [11]:
data[data['is_question'] == False].groupby(['stars'])['text'].count()

stars
1     2264
2     2269
3     4530
4    14235
5    39798
Name: text, dtype: int64

Considering the fact that distribution is still good, I'll filter out questions for now, although there might be some interesting questions in 1th class 
like 
```
Працювали поки був заряджний акумулятор.НЕ ВМИКАЮТЬСЯ.Як оформити поврнення?
```


In [23]:
data[data['text'].str.contains(':\(')]['text'].head(10).values

array(['Появилася тріщина на одній із шкал через 4 роки використання ТЕЧЕ:( Достоинства: Швидко нагріває. Недостатки: Шумить, неякійсний пластик на шкалах',
       'очень мощный комбайн, соковыжималка супер, взбивание коктейлей супер, но нарезка овощей тут конечно подкачало....режет не равномерно, нашинковать огурец невозможно, комбайн его захватывает и по кругу тащит, т.к. от насадки до крышки много места, на штуке, которой толкаешь овощи в комбайн нет зубчиков чтоб зафиксировать овощ, в этом плане не продумано совсем, очень расстроил этот момент, т.к. в основном из-за этого его и брала, очень люблю овощные салаты...за помидоры вообще молчу, делает из них просто пюре и опять же большие куски остаются между насадкой и крышкой.... ну это ладно, помидоры то понятно, а вот за огурцы и все что имеет продолговатой формы (крабовые палочки допустим тоже не получается нарезать красиво) обидно...:(( и даже специальное отделение в штуке, которая проталкивает овощи в комбайн не помогает...(очень 

In [42]:
data[data['text'].str.contains("=\)")]['text'].head(10).values

array(['пользуемся полтора года пока работает отлично =)',
       'Купил этот миксер и теперь благодаря ему и мама и сестра довольно часто им пользуются(не ложкой мешать).Всё хорошо взбивает и готовка в радость.Сам миксер достаточно мощный,а кнопка турбо режима=пятой скорости.=)В общем всем советую.#мояраспаковка Достоинства: Мощный,классные венчики. Недостатки: Тяжело нажимается кнопка Eject.',
       'Не поганий міксер.Купив 1 тиждень назад і не жалію. Тихо працює а креми до тортиків такі пухкі получаються, що аж пальчики оближеш. Блендером пюре класно робиться . Досить зручний в використанні . РАджу купляти прилади від БОШ=)',
       'Заказал для подарка... Жена довольна - я доволен =). Спасибо! Достоинства: Соотношение цена/качество. Мощность. Недостатки: Нож мини-измельчителя не всегда поднимает продукт.',
       'Духовкою ми дуже задоволені! Для своєї ціни виконана дуже якісно і виглядає дуже стильно.  Виник ще нюанс при монтажі у мебель, духовка має знизу підвищення вилиті в ниж